# Imports

In [17]:
import numpy as np
import pandas as pd
import os
import zipfile
import re

# Download and extract the data

In [4]:
if('blogs.zip' not in os.listdir()):
    !curl 'http://u.cs.biu.ac.il/~koppel/blogs/blogs.zip' -H 'Connection: keep-alive' -H 'Upgrade-Insecure-Requests: 1' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'Referer: http://u.cs.biu.ac.il/~koppel/BlogCorpus.htm' -H 'Accept-Encoding: gzip, deflate' -H 'Accept-Language: en-US,en;q=0.9' --compressed --output blogs.zip
    with zipfile.ZipFile('blogs.zip', 'r') as zip_ref:
        zip_ref.extractall('./')

# Data cleaning and exploration

In [43]:
bloggers = os.listdir('blogs/')
print("There are {} bloggers".format(len(bloggers)))

there are 19320 bloggers


In [38]:
path = 'blogs/'

f = open(path+bloggers[0],'r',encoding="utf-8")

xml = f.read()

tags = set()
l = re.findall('<[^/].*?>',xml)
for el in l:
    tags.add(el)
print("The set of tags in the xml files are: {}".format(tags))

the set of tags in the xml files are: {'<date>', '<post>', '<Blog>'}


In [71]:
idx = []
gender = []
age = []
industry = []
astrologic = []
dates = []
posts = []

for blogger in bloggers:
    
    try:

        f = open(path+blogger,'r',encoding="utf-8")
        xml = f.read()
        
    except UnicodeDecodeError:
        
        f = open(path+blogger,'r',encoding="latin-1")
        xml = f.read()
    
    posts += re.findall('<post.*?>(.*?)</post>',xml,flags=re.DOTALL)
    
    dates += re.findall('<date.*?>(.*?)</date>',xml,flags=re.DOTALL)
    
    nb_posts = len(re.findall('<post.*?>(.*?)</post>',xml,flags=re.DOTALL))
    
    tmp = blogger.split(".")
    
    idx        += [tmp[0]]*nb_posts
    gender     += [tmp[1]]*nb_posts
    age        += [tmp[2]]*nb_posts
    industry   += [tmp[3]]*nb_posts
    astrologic += [tmp[4]]*nb_posts

In [73]:
dico = {'id':idx,
        'gender':gender,
        'age':age,
        'industry':industry,
        'astrologic':astrologic,
        'post':posts,
        'date':dates}

data = pd.DataFrame.from_dict(dico)

In [75]:
data.describe()

,id,gender,age,industry,astrologic,post,date
count,681288,681288,681288,681288,681288,681288,681288
unique,19320,2,26,40,12,612135,2616
top,449628,male,17,indUnk,Cancer,\n\n\t \n urlLink \n,"02,August,2004"
freq,4221,345197,80859,251015,65048,399,16545
